In [1]:
import os
os.chdir('./../')

In [2]:
import joblib
import numpy as np
import pandas as pd
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import log_loss
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import PowerTransformer

<frozen importlib._bootstrap>:219: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


In [3]:
all_df = pd.read_csv('./data/train.csv', index_col=0)

In [4]:
x_train, x_test, y_train, y_test = train_test_split(all_df.loc[:, all_df.columns != 'Class'], 
                                                    all_df.loc[:, 'Class'], 
                                                    random_state=42)

In [5]:
index_of_ej = list(x_train.columns).index('EJ')

In [6]:
class CategoricalTransformer:
    
    def __init__(self, index_of_col):
        self._index_of_col = index_of_col
        self._a = None
        self._b = None
    
    def fit(self, x, y):
        if isinstance(x, pd.DataFrame):
            x = x.values
        self._a = y[x[:, self._index_of_col] == 'A'].mean()
        self._b = y[x[:, self._index_of_col] == 'B'].mean()
    
    def transform(self, x):
        if isinstance(x, pd.DataFrame):
            x = x.values
        x[:, self._index_of_col] = np.where(x[:, self._index_of_col] == 'A', self._a, self._b)
        return x
        
    def fit_transform(self, x, y):
        self.fit(x, y)
        return self.transform(x)

In [7]:
pipeline = make_pipeline(CategoricalTransformer(index_of_ej), 
                        IterativeImputer(initial_strategy='median'),
                        PowerTransformer(),
                        LogisticRegression())

In [8]:
pipeline.fit(x_train.values, y_train)

/anaconda/envs/azureml_py38_PT_and_TF/lib/python3.8/site-packages/sklearn/preprocessing/_data.py:3253: RuntimeWarning: divide by zero encountered in log
  loglike = -n_samples / 2 * np.log(x_trans.var())


Pipeline(steps=[('categoricaltransformer',
                 <__main__.CategoricalTransformer object at 0x7f9ddadf29a0>),
                ('iterativeimputer',
                 IterativeImputer(initial_strategy='median')),
                ('powertransformer', PowerTransformer()),
                ('logisticregression', LogisticRegression())])

In [9]:
log_loss(y_train, pipeline.predict_proba(x_train))

0.16008022602499214